In [14]:
## importing the libs
import numpy as np
import pandas as pd
import tensorflow as tf

In [15]:
# importing the data from the file:
class DataReader:
    def __init__(self, path, seq_length):
        #uncomment below , if you dont want to use any file for text reading and comment next 2 lines
        #self.data = "some really long text to test this. maybe not perfect but should get you going."
        self.fp = open(path, "r")
        self.data = self.fp.read()
        #find unique chars
        chars = list(set(self.data))
        #create dictionary mapping for each char
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        #total data
        self.data_size = len(self.data)
        #num of unique chars
        self.vocab_size = len(chars)
        self.pointer = 0
        self.seq_length = seq_length

    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.seq_length
        if self.pointer + self.seq_length + 1 >= self.data_size:
            # reset pointer
            self.pointer = 0
        return inputs, targets

    def just_started(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()

In [35]:
def sigmoid(t):
    return 1/(1+np.exp(-t))

def numpy_to_tensor(something):

    something = tf.convert_to_tensor(something, dtype=tf.float32)
    return something

class RNN:
    
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
    
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate

        # model parameters
        self.U = tf.Variable(numpy_to_tensor(np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))))
        self.V = tf.Variable(numpy_to_tensor(np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))))
        self.W = tf.Variable(numpy_to_tensor(np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))))
        self.b = tf.Variable(numpy_to_tensor(np.zeros((hidden_size, 1)))) # bias for hidden layer
        self.c = tf.Variable(numpy_to_tensor(np.zeros((vocab_size, 1)))) # bias for output
        
        
    def softmax(self, logits):
        softmax=tf.keras.activations.softmax(x, axis=1)
        return softmax
    
    
    def train(self, data_reader):
        loss_iter=[]
        iter_num = 0
        threshold = 0.01
        optimizer = tf.keras.optimizers.Adam()
        
        self.smooth_loss= tf.math.log(1.0/data_reader.vocab_size, name=None)*self.seq_length*-1
        loss_iter.append(self.smooth_loss)
        
        while(self.smooth_loss > threshold):
            if data_reader.just_started():
                hprev=tf.zeros([self.hidden_size,1], tf.float32)
#                 hprev = np.zeros((self.hidden_size,1))
            

            inputs, targets = data_reader.next_batch()
            # doing a forward prop:
            with tf.GradientTape() as tape:
                # initializing the forward parameters through time:
                tape.watch([self.U, self.V, self.W, self.b, self.c])
                xs, hs, os, ycap = {}, {}, {}, {}
                hs[-1] = tf.identity(hprev, name=None)
                
                for t in range(len(inputs)):
                    xs[t] = tf.zeros((self.vocab_size,1))
                    
                    xs[t][inputs[t]] = tf.Variable(1.0)
                    hs[t] = tf.math.tanh(tf.tensordot(self.U,xs[t]) + tf.tensordot(self.W,hs[t-1])+self.b) # hidden state
                    os[t] = tf.tensordot(self.V,hs[t]) + self.c # unnormalised log probs for next char
                    ycap[t] = self.softmax(os[t]) # attaching the softmax of the predicted values
                    
                    
                #updating the loss
                self.reduced_loss = tf.reduce_sum(-1*tf.math.log(ycap[t][targets[t],0]) for t in range(self.seq_length))
                self.smooth_loss = smooth_loss*0.999 + self.reduced_loss*0.001    
#                 return xs, hs, ycap, smooth_loss

            
            loss_iter.append(self.smooth_loss)

            #doing the back prop with the gradient tape:
            grad = tape.gradient(self.reduced_loss, [self.U, self.V, self.W, self.b, self.c])

            optimizer.apply_gradients(zip(grad,[self.U, self.V, self.W, self.b, self.c]))
            
            if not iter_num%500:
                print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
            iter_num += 1
            
            
            
    def predict(self, data_reader, start, n):
        

        #initialize input vector
        x = numpy_to_tensor(np.zeros((self.vocab_size, 1)))
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = numpy_to_tensor(tf.zeros((self.hidden_size,1)))
        # predict next n chars
        for t in range(n):
            h = tf.math.tanh(tf.tensordot(self.U, x) + tf.tensordot(self.W, h) + self.b)
            y = tf.tensordot(self.V, h) + self.c
            p = tf.math.exp(y)/tf.reduce_sum(tf.math.exp(y))
            
            ### converting p to numpy again:
            p_n=p.numpy()
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = numpy_to_tensor(np.zeros((self.vocab_size,1)))
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt
            
            
    

In [36]:
seq_length = 6
#read text from the "input.txt" file
data_reader = DataReader("temp.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
loss=rnn.train(data_reader)

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment